In [1]:
from transformers import GPT2Tokenizer, LineByLineTextDataset, DataCollatorForLanguageModeling, GPT2LMHeadModel, TrainingArguments, Trainer
import torch 

# Initialize the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

# Define the size of the text blocks
block_size = 128

# Load the dataset
train_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="medical_corpus.txt",
    block_size=block_size,
)

# DataCollator handles padding
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, 
    mlm=False,
)

# Initialize the model
torch.set_default_dtype(torch.half)
model = GPT2LMHeadModel.from_pretrained('gpt2')
torch.set_default_dtype(torch.float)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=16,
    save_steps=100,
    save_total_limit=2,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

# Training
#trainer.train()


/home/minigpt_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/minigpt_env/lib/python3.9/site-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
from transformers import GPT2Tokenizer, LineByLineTextDataset, DataCollatorForLanguageModeling, GPT2LMHeadModel
from transformers import TrainingArguments, Trainer, DeepSpeedPlugin, DeepSpeedConfig, Accelerator
import torch
import deepspeed

# Initialize the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

# Define the size of the text blocks
block_size = 128

# Load the dataset
train_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="medical_corpus.txt",
    block_size=block_size,
)

# DataCollator handles padding
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, 
    mlm=False,
)

# Initialize the model
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=16,
    save_steps=100,
    save_total_limit=2,
    deepspeed="/home/medical-llama/ds_config.json",  # path to deepspeed configuration file
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

# Training
trainer.train()


In [ ]:
from transformers import GPT2Tokenizer, LineByLineTextDataset, DataCollatorForLanguageModeling, GPT2LMHeadModel, TrainingArguments, Trainer
import torch 
from accelerate import Accelerator

accelerator = Accelerator()

# Initialize the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

# Define the size of the text blocks
block_size = 128

# Load the dataset
train_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="medical_corpus.txt",
    block_size=block_size,
)

# DataCollator handles padding
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, 
    mlm=False,
)

# Initialize the model
torch.set_default_dtype(torch.half)
model = GPT2LMHeadModel.from_pretrained('gpt2')
torch.set_default_dtype(torch.float)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=16,
    save_steps=100,
    save_total_limit=2,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    accelerator=accelerator,
)

# Training
trainer.train()


In [5]:
from transformers import AdamW
import torch
from torch.cuda.amp import GradScaler, autocast
from torch.nn.utils.rnn import pad_sequence

# Initialize the optimizer
optimizer = AdamW(model.parameters(),lr=0.0001)

# For fp16 training
scaler = GradScaler()

# Define device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Move model to device
model.to(device)

# Convert dataset to DataLoader
from torch.utils.data import DataLoader

max_seq_length = 10

# Pad tokenized texts
def pad_collate_fn(batch):
    padded_batch = pad_sequence([torch.tensor(text) for text in batch], batch_first=True, padding_value=0)
    return padded_batch


dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16, collate_fn=pad_collate_fn)

# Training loop
model.train()
for epoch in range(10):  # num of epochs
    for i, batch in enumerate(dataloader):
        # Move batch to device
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass under autocast
        with autocast():
            outputs = model(**batch)
            loss = outputs.loss

        # Backward pass with gradient scaling
        scaler.scale(loss).backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Optimizer step and update scale
        scaler.step(optimizer)
        scaler.update()

        # Zero the gradients
        optimizer.zero_grad()

        # Log the loss
        if i % 10 == 0:  # print every 100 batches
            print(f'Epoch: {epoch}, Batch: {i}, Loss: {loss.item()}')
            
    # Save the model after each epoch
    #torch.save(model.state_dict(), f'./results/model_{epoch}.bin')


/home/minigpt_env/lib/python3.9/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:34                                                                                   │
│                                                                                                  │
│   31 # Training loop                                                                             │
│   32 model.train()                                                                               │
│   33 for epoch in range(10):  # num of epochs                                                    │
│ ❱ 34 │   for i, batch in enumerate(dataloader):                                                  │
│   35 │   │   # Move batch to device                                                              │
│   36 │   │   batch = {k: v.to(device) for k, v in batch.items()}                                 │
│   37                                                                                             │
│                                                                                                  │
│ /home/minigpt_env/lib/python3.9/site-packages/torch/utils/data/dataloader.py:633 in __next__     │
│                                                                                                  │
│    630 │   │   │   if self._sampler_iter is None:                                                │
│    631 │   │   │   │   # TODO(https://github.com/pytorch/pytorch/issues/76750)                   │
│    632 │   │   │   │   self._reset()  # type: ignore[call-arg]                                   │
│ ❱  633 │   │   │   data = self._next_data()                                                      │
│    634 │   │   │   self._num_yielded += 1                                                        │
│    635 │   │   │   if self._dataset_kind == _DatasetKind.Iterable and \                          │
│    636 │   │   │   │   │   self._IterableDataset_len_called is not None and \                    │
│                                                                                                  │
│ /home/minigpt_env/lib/python3.9/site-packages/torch/utils/data/dataloader.py:677 in _next_data   │
│                                                                                                  │
│    674 │                                                                                         │
│    675 │   def _next_data(self):                                                                 │
│    676 │   │   index = self._next_index()  # may raise StopIteration                             │
│ ❱  677 │   │   data = self._dataset_fetcher.fetch(index)  # may raise StopIteration              │
│    678 │   │   if self._pin_memory:                                                              │
│    679 │   │   │   data = _utils.pin_memory.pin_memory(data, self._pin_memory_device)            │
│    680 │   │   return data                                                                       │
│                                                                                                  │
│ /home/minigpt_env/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py:54 in fetch       │
│                                                                                                  │
│   51 │   │   │   │   data = [self.dataset[idx] for idx in possibly_batched_index]                │
│   52 │   │   else:                                                                               │
│   53 │   │   │   data = self.dataset[possibly_batched_index]                                     │
│ ❱ 54 │   │   return self.collate_fn(data)                                                        │
│   55                                                                                             │
│                                                                                                  │
│ in pad_collate_fn:25                                       

In [7]:
dataloader

In [6]:
for i in dataloader:
    print(i)
    break   

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 for i in dataloader:                                                                         │
│   2 │   print(i)                                                                                 │
│   3 │   break                                                                                    │
│   4                                                                                              │
│                                                                                                  │
│ /home/minigpt_env/lib/python3.9/site-packages/torch/utils/data/dataloader.py:633 in __next__     │
│                                                                                                  │
│    630 │   │   │   if self._sampler_iter is None:                                                │
│    631 │   │   │   │   # TODO(https://github.com/pytorch/pytorch/issues/76750)                   │
│    632 │   │   │   │   self._reset()  # type: ignore[call-arg]                                   │
│ ❱  633 │   │   │   data = self._next_data()                                                      │
│    634 │   │   │   self._num_yielded += 1                                                        │
│    635 │   │   │   if self._dataset_kind == _DatasetKind.Iterable and \                          │
│    636 │   │   │   │   │   self._IterableDataset_len_called is not None and \                    │
│                                                                                                  │
│ /home/minigpt_env/lib/python3.9/site-packages/torch/utils/data/dataloader.py:677 in _next_data   │
│                                                                                                  │
│    674 │                                                                                         │
│    675 │   def _next_data(self):                                                                 │
│    676 │   │   index = self._next_index()  # may raise StopIteration                             │
│ ❱  677 │   │   data = self._dataset_fetcher.fetch(index)  # may raise StopIteration              │
│    678 │   │   if self._pin_memory:                                                              │
│    679 │   │   │   data = _utils.pin_memory.pin_memory(data, self._pin_memory_device)            │
│    680 │   │   return data                                                                       │
│                                                                                                  │
│ /home/minigpt_env/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py:54 in fetch       │
│                                                                                                  │
│   51 │   │   │   │   data = [self.dataset[idx] for idx in possibly_batched_index]                │
│   52 │   │   else:                                                                               │
│   53 │   │   │   data = self.dataset[possibly_batched_index]                                     │
│ ❱ 54 │   │   return self.collate_fn(data)                                                        │
│   55                                                                                             │
│                                                                                                  │
│ in pad_collate_fn:25                                                                             │
│                                                                                                  │
│   22                                                                                             │
│   23 # Pad tokenized texts                                 